# DidgeLab Conversion Utilities (`didgelab.conv`)

This notebook documents the musical note and frequency conversion utilities in `didgelab.conv`. These functions convert between:

- **Note numbers** (relative to a base frequency, e.g. A4 = 0)
- **Note names** (e.g. "A4", "B4", "C#5")
- **Frequencies** in Hz
- **Wavelengths** in mm (at 343.2 m/s speed of sound)
- **Cent differences** between two frequencies

All functions accept **dual parameters**: arguments can be Python scalars or numpy arrays. The return type matches the input (scalar in → scalar out, array in → array out). Broadcasting applies when multiple array arguments are used.

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
from didgelab import (
    note_to_freq,
    note_name,
    freq_to_note,
    freq_to_note_and_cent,
    freq_to_wavelength,
    note_name_to_number,
    cent_diff,
)

## 1. Note number ↔ Frequency

Note numbers are semitones relative to a base frequency (default 440 Hz = A4). 12 semitones = 1 octave.

In [ ]:
# Scalar usage
print("note_to_freq(0, 440) =", note_to_freq(0, 440), "Hz  # A4")
print("note_to_freq(12, 440) =", note_to_freq(12, 440), "Hz  # A5 (one octave up)")
print("note_to_freq(-12, 440) =", note_to_freq(-12, 440), "Hz  # A3 (one octave down)")
print()
print("freq_to_note(440, 440) =", freq_to_note(440, 440), "  # A4")
print("freq_to_note(880, 440) =", freq_to_note(880, 440), "  # A5")
print("freq_to_note(220, 440) =", freq_to_note(220, 440), "  # A3")

### Array usage

Functions accept numpy arrays and return arrays of the same shape.

In [ ]:
notes = np.array([0, 3, 12, -9])  # A4, C5, A5, C4
freqs = note_to_freq(notes, 440)
print("Notes:", notes)
print("Frequencies (Hz):", freqs)
print()
# Round-trip
back = freq_to_note(freqs, 440)
print("Round-trip freq_to_note:", back)

## 2. Note number ↔ Note name

Convert between semitone numbers and note names like "A4", "C#5".

In [ ]:
print("note_name(0) =", note_name(0))
print("note_name(12) =", note_name(12))
print("note_name(-9) =", note_name(-9), "  # C4")
print()
print("note_name_to_number('A4') =", note_name_to_number("A4"))
print("note_name_to_number('C#5') =", note_name_to_number("C#5"))

In [ ]:
# Arrays of note names
names = np.array(["A4", "A#4", "B4", "C5"])
nums = note_name_to_number(names)
print("Note names:", names)
print("Note numbers:", nums)
print("Round-trip:", note_name(nums))

## 3. Frequency → Note and cents

`freq_to_note_and_cent` returns how many semitones and cents a frequency is from the nearest note. Useful for tuning analysis.

In [ ]:
freq = 438  # Slightly flat A4
note, cents = freq_to_note_and_cent(freq, 440)
print(f"{freq} Hz → note {note} ({note_name(note)}), {cents:.2f} cents from in-tune")
print()
freq = 466.16  # A#4
note, cents = freq_to_note_and_cent(freq, 440)
print(f"{freq} Hz → note {note} ({note_name(note)}), {cents:.2f} cents")

## 4. Wavelength

`freq_to_wavelength` returns wavelength in mm (speed of sound = 343.2 m/s).

In [ ]:
print("343.2 Hz → wavelength =", freq_to_wavelength(343.2), "mm (= 1 m)")
print("100 Hz (typical drone) → wavelength =", freq_to_wavelength(100), "mm")

## 5. Cent difference

`cent_diff(freq1, freq2)` returns how many cents freq2 is above freq1. 100 cents = 1 semitone, 1200 cents = 1 octave.

In [ ]:
print("cent_diff(440, 880) =", cent_diff(440, 880), "cents  # one octave up")
print("cent_diff(440, 440*2**(1/12)) =", cent_diff(440, 440*2**(1/12)), "cents  # one semitone")
print("cent_diff(440, 438) ≈", round(cent_diff(440, 438), 2), "cents  # slightly flat")

## 6. Practical example: Map impedance peak to note

Typical use in DidgeLab: an impedance spectrum has a peak at ~73 Hz (drone). Convert that to a note name and cent deviation.

In [ ]:
drone_freq = 73.42
note_num, cents = freq_to_note_and_cent(drone_freq, 440)
name = note_name(note_num)
sign = "+" if cents >= 0 else ""
print(f"Drone at {drone_freq} Hz → {name} ({sign}{cents:.2f} cents)")

## 7. Alternative tuning (e.g. 432 Hz)

Pass a custom `base_freq` for non-440 Hz reference.

In [ ]:
print("With base_freq=440:", note_to_freq(0, 440), "Hz")
print("With base_freq=432:", note_to_freq(0, 432), "Hz")
print("Difference in cents:", cent_diff(432, 440), "cents")